## KY dimension for a single module

In [13]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from itertools import product

inps = [0.1, 1.0, 10.0]
hids = [2, 10]
rhos = [0.5, 0.9, 10.0]

kyresults = {}
for inp_scaling, n_hid, rho in product(inps, hids, rhos):
    print(f"inp_scaling: {inp_scaling}, n_hid: {n_hid}, rho: {rho}")
    directory = f'/scratch/a.cossu/results_single/rho_{rho}_nhid_{n_hid}_inp_scaling_{inp_scaling}_timesteps_3000'
    seq_len = 2000  # timesteps - washout
    n_init_states = 1000
    n_inp = 1


    trajectories = np.load(os.path.join(directory, 'all_states.npy'))  # (n_init_states, timesteps, hidden_size)
    filename = "lyapunov_rnn.dat"

    input_signals = np.load(os.path.join(directory, 'u_timeseries.npy'))

    # move files from the saved directory to the current directory
    shutil.move(os.path.join(directory, "W.csv"), "W.csv")
    shutil.move(os.path.join(directory, "V.csv"), "V.csv")
    shutil.move(os.path.join(directory, "b.csv"), "b.csv")

    kds = []
    for i, traj in enumerate(tqdm(trajectories)):
        input_signal = np.expand_dims(input_signals[i], axis=-1) # (seq_len, 1)
        np.savetxt("u_timeseries.csv", input_signal, delimiter=",", fmt="%.6f")
        np.savetxt("h_traj.csv", traj, delimiter=',', fmt="%.6f")

        # call rnn_lyap with parameters n_hid, seq_len, n_inp
        os.system(f"./rnn_lyap {n_hid} {seq_len} {n_inp}")

        data = np.loadtxt(filename)
        # Extract time (first column) and exponents (remaining columns)
        time = data[:, 0]
        lyapunov_exponents = data[:, 1:]


        # Plot each Lyapunov exponent as a function of time
        # plt.figure(figsize=(10, 6))
        # # plot the zero line for reference in black dashed line
        # plt.plot(time, np.zeros_like(time), color='black', linestyle='--', linewidth=1)
        # for i in range(lyapunov_exponents.shape[1]):
        #     plt.plot(time, lyapunov_exponents[:, i], label=f"LE {i+1}")

        # plt.xlabel("Time")
        # plt.ylabel("Lyapunov Exponents")
        # plt.title("Convergence of Lyapunov Exponents in RNN")
        # plt.legend()
        # plt.grid(True, linestyle='--', alpha=0.6)
        # plt.tight_layout()
        # plt.show()

        best_approx_lyaps = lyapunov_exponents[-1, :]
        # print(best_approx_lyaps)

        # Example: lyapunov_exponents.shape is (N,)
        # Make sure the spectrum is sorted in descending order
        best_approx_lyaps = np.sort(best_approx_lyaps)[::-1]

        if np.any(best_approx_lyaps > 0):
            # Cumulative sum
            cumsum = np.cumsum(best_approx_lyaps)

            # Find j: largest index where cumulative sum is still >= 0
            j = np.where(cumsum >= 0)[0][-1]  # last index satisfying sum >= 0

            # print("j (number of non-negative LEs):", j + 1)

            # Compute Kaplan-Yorke dimension
            if j + 1 < len(best_approx_lyaps):
                D_KY = j + 1 + cumsum[j] / abs(best_approx_lyaps[j + 1])
            else:
                # All exponents non-negative 
                D_KY = len(best_approx_lyaps)
        else:
            # All exponents are non-positive
            D_KY = 0
        # print("******************")
        #print("Kaplan-Yorke dimension:", D_KY)
        # print("******************")
        kds.append(D_KY)

    print(f"Kaplan-Yorke dimension over trajectories: {np.mean(kds)} ± {np.std(kds)}")
    max_idx = np.argmax(kds)
    min_idx = np.argmin(kds)
    print(f"Max Kaplan-Yorke at index {max_idx} with value {kds[max_idx]}")
    print(f"Min Kaplan-Yorke at index {min_idx} with value {kds[min_idx]}")

    # move files back to the saved directory
    shutil.move("W.csv", os.path.join(directory, "W.csv"))
    shutil.move("V.csv", os.path.join(directory, "V.csv"))
    shutil.move("b.csv", os.path.join(directory, "b.csv"))

    kyresults[(inp_scaling, n_hid, rho)] = kds

inp_scaling: 0.1, n_hid: 2, rho: 0.5


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:10<00:00, 91.72it/s]


Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
inp_scaling: 0.1, n_hid: 2, rho: 0.9


100%|██████████| 1000/1000 [00:11<00:00, 90.52it/s]


...

Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
inp_scaling: 10.0, n_hid: 10, rho: 0.9


100%|██████████| 1000/1000 [00:13<00:00, 72.12it/s]


Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
inp_scaling: 10.0, n_hid: 10, rho: 10.0


100%|██████████| 1000/1000 [00:16<00:00, 62.27it/s]

Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0


In [14]:
for k, v in kyresults.items():
    inp_scaling, n_hid, rho = k
    print(f"inp_scaling={inp_scaling}, n_hid={n_hid}, rho={rho}: KD mean={np.mean(v)} ± std={np.std(v)}, max={np.max(v)}, min={np.min(v)}")

    # plot histogram
    dir = f'/scratch/a.cossu/results_single/rho_{rho}_nhid_{n_hid}_inp_scaling_{inp_scaling}_timesteps_3000'
    plt.figure()
    plt.hist(kyresults[(inp_scaling, n_hid, rho)], bins=30)
    plt.savefig(os.path.join(dir, "ky_histogram.png"))
    plt.close()

inp_scaling=0.1, n_hid=2, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=0.1, n_hid=2, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=0.1, n_hid=2, rho=10.0: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=0.1, n_hid=10, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=0.1, n_hid=10, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=0.1, n_hid=10, rho=10.0: KD mean=0.43198965536242784 ± std=0.5358651632698999, max=1.1564159450925897, min=0.0
inp_scaling=1.0, n_hid=2, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=1.0, n_hid=2, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=1.0, n_hid=2, rho=10.0: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=1.0, n_hid=10, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=1.0, n_hid=10, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
inp_scaling=1.0, n_hid=10, rho=10.0: KD mean=0.42510915503891306 ± std=0.5278199391677835, max=1.1558690581719193, min=0.0
inp_scaling=10.0, n_hid=2, rho=0.5: KD

## KY dimension for a collective of modules

In [15]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import shutil
from itertools import product

inps = [0.1, 1.0, 10.0]
hids = [2, 10]
rhos = [0.5, 0.9, 10.0]

n_modules = 2

kyresults = [{} for _ in range(n_modules)]

for inp_scaling, n_hid, rho in product(inps, hids, rhos):
    print(f"inp_scaling: {inp_scaling}, n_hid: {n_hid}, rho: {rho}")
    directory = f'/scratch/a.cossu/results_collective/mod2_rho_{rho}_nhid_{n_hid}_timesteps_3000_inpscaling_{inp_scaling}'
    seq_len = 2000  # timesteps - washout
    n_init_states = 1000
    n_inp = n_hid

    input_signal = torch.load(os.path.join(directory, f"input_signals.pt"))

    for i in range(n_modules):
        print(f"Processing module {i}")
        trajectories = np.load(os.path.join(directory, f'all_states{i}.npy')) 

        filename = "lyapunov_rnn.dat"

        # move files from the saved directory to the current directory
        shutil.move(os.path.join(directory, f"W_{i}.csv"), f"W.csv")
        shutil.move(os.path.join(directory, f"V_{i}.csv"), f"V.csv")
        shutil.move(os.path.join(directory, f"b_{i}.csv"), f"b.csv")

        kds = []
        for t, traj in enumerate(tqdm(trajectories)):
            np.savetxt(os.path.join("u_timeseries.csv"), input_signal[i][t].cpu().numpy(), delimiter=",", fmt="%.6f")
            np.savetxt("h_traj.csv", traj, delimiter=',', fmt="%.6f")

            # call rnn_lyap with parameters n_hid, seq_len, n_inp
            os.system(f"./rnn_lyap {n_hid} {seq_len} {n_inp}")

            data = np.loadtxt(filename)
            # Extract time (first column) and exponents (remaining columns)
            time = data[:, 0]
            lyapunov_exponents = data[:, 1:]


            # Plot each Lyapunov exponent as a function of time
            # plt.figure(figsize=(10, 6))
            # # plot the zero line for reference in black dashed line
            # plt.plot(time, np.zeros_like(time), color='black', linestyle='--', linewidth=1)
            # for i in range(lyapunov_exponents.shape[1]):
            #     plt.plot(time, lyapunov_exponents[:, i], label=f"LE {i+1}")

            # plt.xlabel("Time")
            # plt.ylabel("Lyapunov Exponents")
            # plt.title("Convergence of Lyapunov Exponents in RNN")
            # plt.legend()
            # plt.grid(True, linestyle='--', alpha=0.6)
            # plt.tight_layout()
            # plt.show()

            best_approx_lyaps = lyapunov_exponents[-1, :]
            # print(best_approx_lyaps)

            # Example: lyapunov_exponents.shape is (N,)
            # Make sure the spectrum is sorted in descending order
            best_approx_lyaps = np.sort(best_approx_lyaps)[::-1]

            if np.any(best_approx_lyaps > 0):
                # Cumulative sum
                cumsum = np.cumsum(best_approx_lyaps)

                # Find j: largest index where cumulative sum is still >= 0
                j = np.where(cumsum >= 0)[0][-1]  # last index satisfying sum >= 0

                # print("j (number of non-negative LEs):", j + 1)

                # Compute Kaplan-Yorke dimension
                if j + 1 < len(best_approx_lyaps):
                    D_KY = j + 1 + cumsum[j] / abs(best_approx_lyaps[j + 1])
                else:
                    # All exponents non-negative 
                    D_KY = len(best_approx_lyaps)
            else:
                # All exponents are non-positive
                D_KY = 0
            # print("******************")
            #print("Kaplan-Yorke dimension:", D_KY)
            # print("******************")
            kds.append(D_KY)

        print(f"Kaplan-Yorke dimension over trajectories: {np.mean(kds)} ± {np.std(kds)}")
        max_idx = np.argmax(kds)
        min_idx = np.argmin(kds)
        print(f"Max Kaplan-Yorke at index {max_idx} with value {kds[max_idx]}")
        print(f"Min Kaplan-Yorke at index {min_idx} with value {kds[min_idx]}")
        
        # move files back to the saved directory
        shutil.move("W.csv", os.path.join(directory, f"W_{i}.csv"))
        shutil.move("V.csv", os.path.join(directory, f"V_{i}.csv"))
        shutil.move("b.csv", os.path.join(directory, f"b_{i}.csv"))
        kyresults[i][(inp_scaling, n_hid, rho)] = kds
        print("=========================================")

inp_scaling: 0.1, n_hid: 2, rho: 0.5
Processing module 0


100%|██████████| 1000/1000 [00:12<00:00, 82.49it/s]


Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
Processing module 1


100%|██████████| 1000/1000 [00:12<00:00, 82.78it/s]


Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
inp_scaling: 0.1, n_hid: 2, rho: 0.9
Processing module 0


...

Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
inp_scaling: 10.0, n_hid: 10, rho: 10.0
Processing module 0


100%|██████████| 1000/1000 [00:16<00:00, 58.88it/s]


Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0
Processing module 1


100%|██████████| 1000/1000 [00:16<00:00, 59.89it/s]

Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Max Kaplan-Yorke at index 0 with value 0
Min Kaplan-Yorke at index 0 with value 0


In [16]:
for m, kyd in enumerate(kyresults):  # for each module
    for k, v in kyd.items():
        inp_scaling, n_hid, rho = k
        print(f"Module {m} inp_scaling={inp_scaling}, n_hid={n_hid}, rho={rho}: KD mean={np.mean(v)} ± std={np.std(v)}, max={np.max(v)}, min={np.min(v)}")

        # plot histogram
        dir = f'/scratch/a.cossu/results_collective/mod2_rho_{rho}_nhid_{n_hid}_timesteps_3000_inpscaling_{inp_scaling}'
        plt.figure()
        plt.hist(kyd[(inp_scaling, n_hid, rho)], bins=30)
        plt.savefig(os.path.join(dir, f"ky_histogram_mod{m}.png"))
        plt.close()

Module 0 inp_scaling=0.1, n_hid=2, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=0.1, n_hid=2, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=0.1, n_hid=2, rho=10.0: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=0.1, n_hid=10, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=0.1, n_hid=10, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=0.1, n_hid=10, rho=10.0: KD mean=0.24836980438745454 ± std=0.4395684170151863, max=1.0310220534371826, min=0.0
Module 0 inp_scaling=1.0, n_hid=2, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=1.0, n_hid=2, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=1.0, n_hid=2, rho=10.0: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=1.0, n_hid=10, rho=0.5: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=1.0, n_hid=10, rho=0.9: KD mean=0.0 ± std=0.0, max=0, min=0
Module 0 inp_scaling=1.0, n_hid=10, rho=10.0: KD mean=0.0 ± st

## Plot a single trajectory at a time

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
# plot a single trajectory
def plot_trajectory(pca_traj, idx):
    plt.figure()
    plt.scatter(pca_traj[0], pca_traj[1], s=1)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.savefig(f"trajectory{idx}.png")
    plt.close()


directory = '/scratch/a.cossu/results_collective/mod2_rho_10.0_nhid_10_timesteps_3000_inpscaling_0.1'
filename = 'pca_result_1.npy'
seq_len = 2000  # timesteps - washout
plot_how_many = 200  # plot the first plot_how_many trajectories
plot_only = 185 # set to -1 to plot plot_how_many trajectories, or set to a specific index to plot only that trajectory

if plot_only >= 0:
    plot_how_many = plot_only + 1

# load from pca
trajectories = np.load(os.path.join(directory, filename))  # (n_init_states*timesteps, 2)
idx = 0
for k in range(0, trajectories.shape[0], seq_len):
    traj = trajectories[k:k+seq_len]

    if plot_only < 0 or idx == plot_only:
        plot_trajectory(traj, idx=idx)
        if plot_only >= 0:
            break
    idx += 1

    if idx >= plot_how_many:
        break


In [21]:
# remove all trajectory.png 
import os
for file in os.listdir():
    if file.startswith("trajectory") and file.endswith(".png"):
        os.remove(file)

## SKDIM

### Correlation dimension

In [12]:
import os
import numpy as np
import skdim


foldernames = ["results_single", "results_collective"]
for foldername in foldernames:
    for folder in os.listdir(f"/scratch/a.cossu/{foldername}/"):
        folder_path = os.path.join(f"/scratch/a.cossu/{foldername}/", folder)
        if os.path.isdir(folder_path):
            if foldername == "results_collective":
                n_mods = int(folder.split("_")[0].replace("mod", ""))
                corrdims = []
                for i in range(n_mods):
                    all_states = np.load(os.path.join(folder_path, f"all_states{i}.npy"))
                    reshaped_states = all_states.reshape((all_states.shape[0], -1))
                    corr_dim = skdim.id.CorrInt().fit_transform(reshaped_states)
                    corrdims.append(corr_dim)
                print(f"Folder: {folder}, Correlation Dimension: {[float(c) for c in corrdims]}")
                # write corrdims to a file in folder
                with open(os.path.join(folder_path, "correlation_dimensions.txt"), "w") as f:
                    for i, cd in enumerate(corrdims):
                        f.write(f"{i}: {cd}\n")
            else:
                all_states = np.load(os.path.join(folder_path, "all_states.npy"))
                reshaped_states = all_states.reshape((all_states.shape[0], -1))
                corr_dim = skdim.id.CorrInt().fit_transform(reshaped_states)
                print(f"Folder: {folder}, Correlation Dimension: {corr_dim}")
                with open(os.path.join(folder_path, "correlation_dimensions.txt"), "w") as f:
                    f.write(f"{corr_dim}\n")
        else:
            print(f"Skipping {folder_path}, not a directory.")

Folder: rho_0.5_nhid_2_inp_scaling_0.1_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.5_nhid_2_inp_scaling_1.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.5_nhid_2_inp_scaling_10.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.5_nhid_10_inp_scaling_0.1_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.5_nhid_10_inp_scaling_1.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.5_nhid_10_inp_scaling_10.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.9_nhid_2_inp_scaling_0.1_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.9_nhid_2_inp_scaling_1.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.9_nhid_2_inp_scaling_10.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.9_nhid_10_inp_scaling_0.1_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.9_nhid_10_inp_scaling_1.0_timesteps_3000, Correlation Dimension: nan
Folder: rho_0.9_nhid_10_inp_scaling_10.0_timesteps_3000, Correlation Dimension: nan
Folder

### Manifold-Adaptive Dimension

In [ ]:
import os
import numpy as np
import skdim


foldernames = ["results_single", "results_collective"]
for foldername in foldernames:
    for folder in os.listdir(f"/scratch/a.cossu/{foldername}/"):
        folder_path = os.path.join(f"/scratch/a.cossu/{foldername}/", folder)
        if os.path.isdir(folder_path):
            if foldername == "results_collective":
                n_mods = int(folder.split("_")[0].replace("mod", ""))
                corrdims = []
                for i in range(n_mods):
                    all_states = np.load(os.path.join(folder_path, f"all_states{i}.npy"))
                    reshaped_states = all_states.reshape((all_states.shape[0], -1))
                    corr_dim = skdim.id.MADA().fit_transform(reshaped_states)
                    corrdims.append(corr_dim)
                print(f"Folder: {folder}, Manifold Dimension: {[float(c) for c in corrdims]}")
                with open(os.path.join(folder_path, "manifold_dimensions.txt"), "w") as f:
                    for i, cd in enumerate(corrdims):
                        f.write(f"{i}: {cd}\n")
            else:
                all_states = np.load(os.path.join(folder_path, "all_states.npy"))
                reshaped_states = all_states.reshape((all_states.shape[0], -1))
                corr_dim = skdim.id.MADA().fit_transform(reshaped_states)
                print(f"Folder: {folder}, Manifold Dimension: {corr_dim}")
                with open(os.path.join(folder_path, "manifold_dimensions.txt"), "w") as f:
                    f.write(f"{corr_dim}\n")
        else:
            print(f"Skipping {folder_path}, not a directory.")

Folder: rho_0.5_nhid_2_inp_scaling_0.1_timesteps_3000, Manifold Dimension: nan
Folder: rho_0.5_nhid_2_inp_scaling_1.0_timesteps_3000, Manifold Dimension: nan
Folder: rho_0.5_nhid_2_inp_scaling_10.0_timesteps_3000, Manifold Dimension: nan
Folder: rho_0.5_nhid_10_inp_scaling_0.1_timesteps_3000, Manifold Dimension: nan
Folder: rho_0.5_nhid_10_inp_scaling_1.0_timesteps_3000, Manifold Dimension: nan
Folder: rho_0.5_nhid_10_inp_scaling_10.0_timesteps_3000, Manifold Dimension: nan
Folder: rho_0.9_nhid_2_inp_scaling_0.1_timesteps_3000, Manifold Dimension: inf
Folder: rho_0.9_nhid_2_inp_scaling_1.0_timesteps_3000, Manifold Dimension: inf
Folder: rho_0.9_nhid_2_inp_scaling_10.0_timesteps_3000, Manifold Dimension: inf
Folder: rho_0.9_nhid_10_inp_scaling_0.1_timesteps_3000, Manifold Dimension: inf
Folder: rho_0.9_nhid_10_inp_scaling_1.0_timesteps_3000, Manifold Dimension: inf
Folder: rho_0.9_nhid_10_inp_scaling_10.0_timesteps_3000, Manifold Dimension: inf
Folder: rho_10.0_nhid_2_inp_scaling_0.1_ti

### lPCA dimension

In [18]:
import os
import numpy as np
import skdim


foldernames = ["results_single", "results_collective"]
for foldername in foldernames:
    for folder in os.listdir(f"/scratch/a.cossu/{foldername}/"):
        folder_path = os.path.join(f"/scratch/a.cossu/{foldername}/", folder)
        if os.path.isdir(folder_path):
            if foldername == "results_collective":
                n_mods = int(folder.split("_")[0].replace("mod", ""))
                corrdims = []
                for i in range(n_mods):
                    all_states = np.load(os.path.join(folder_path, f"all_states{i}.npy"))
                    reshaped_states = all_states.reshape((all_states.shape[0], -1))
                    corr_dim = skdim.id.lPCA().fit_transform(reshaped_states)
                    corrdims.append(corr_dim)
                print(f"Folder: {folder}, lPCA Dimension: {[float(c) for c in corrdims]}")
                # write corrdims to a file in folder
                with open(os.path.join(folder_path, "lpca_dimensions.txt"), "w") as f:
                    for i, cd in enumerate(corrdims):
                        f.write(f"{i}: {cd}\n")
            else:
                all_states = np.load(os.path.join(folder_path, "all_states.npy"))
                reshaped_states = all_states.reshape((all_states.shape[0], -1))
                corr_dim = skdim.id.lPCA().fit_transform(reshaped_states)
                print(f"Folder: {folder}, lPCA Dimension: {corr_dim}")
                with open(os.path.join(folder_path, "lpca_dimensions.txt"), "w") as f:
                    f.write(f"{corr_dim}\n")
        else:
            print(f"Skipping {folder_path}, not a directory.")

Folder: rho_0.5_nhid_2_inp_scaling_0.1_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.5_nhid_2_inp_scaling_1.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.5_nhid_2_inp_scaling_10.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.5_nhid_10_inp_scaling_0.1_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.5_nhid_10_inp_scaling_1.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.5_nhid_10_inp_scaling_10.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.9_nhid_2_inp_scaling_0.1_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.9_nhid_2_inp_scaling_1.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.9_nhid_2_inp_scaling_10.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.9_nhid_10_inp_scaling_0.1_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.9_nhid_10_inp_scaling_1.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_0.9_nhid_10_inp_scaling_10.0_timesteps_3000, lPCA Dimension: 0
Folder: rho_10.0_nhid_2_inp_scaling_0.1_timesteps_3000, lPCA Dimension: 1
Folder: rho_10.0_nhid_2_inp_scaling_1.0_